In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score

df = pd.read_csv('Datasets/cleaned_data.csv')
#df = pd.read_csv('Datasets/feature_engineered_data.csv')

df

# column_headers_list = df.columns.tolist()
# print(column_headers_list)
# column_b = df['is_weekend']

# # Print the 'B' column
# print(column_b)

,date,id,mood,circumplex.arousal,circumplex.valence,activity,screen,call,sms,appCat.builtin,...,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.weather,appCat.utilities
0,2014-03-04,AS14.17,7.000000,0.500000,1.000000,0.042303,0.000000,5.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
1,2014-03-05,AS14.08,6.333333,0.000000,0.333333,0.005556,0.000000,0.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
2,2014-03-05,AS14.17,8.000000,0.666667,1.333333,0.042303,0.000000,1.0,1.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
3,2014-03-06,AS14.17,7.250000,0.500000,0.750000,0.042303,0.000000,1.0,0.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
4,2014-03-06,AS14.08,7.200000,-0.600000,0.600000,0.005556,0.000000,0.0,13.0,0.000,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,2014-06-05,AS14.24,7.000000,0.000000,0.800000,0.152343,2371.588999,11.0,0.0,1749.277,...,0.0,0.0,0.0,0.0,47.360,171.964,225.737,0.0,0.0,0.000
1301,2014-06-06,AS14.24,7.500000,0.500000,0.250000,0.189476,2733.332000,16.0,0.0,945.898,...,0.0,0.0,0.0,0.0,36.414,0.000,0.000,0.0,0.0,2.072
1302,2014-06-07,AS14.24,8.000000,-1.000000,0.000000,0.081893,5530.377999,0.0,2.0,5027.498,...,0.0,0.0,0.0,0.0,6.214,106.850,0.000,0.0,0.0,0.000
1303,2014-06-08,AS14.24,6.500000,-1.500000,1.000000,0.079510,1131.197999,0.0,0.0,351.389,...,0.0,0.0,0.0,0.0,18.069,190.760,0.000,0.0,0.0,0.000


In [34]:
# Round the mood column to nearest integer
df['mood'] = df['mood'].round()
df['date'] = pd.to_datetime(df['date'])

# Extract year, month, day, hour, and minute as separate columns
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day

# Perform one-hot encoding one the persons
one_hot_df = pd.get_dummies(df['id'])

# Concatenate the one-hot encoded columns to the original DataFrame
df = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical column
df.drop(['id'], axis=1, inplace=True)

In [35]:
# Define the input and target variables
X = df.drop(["date", "mood"], axis=1) # drop the date and target columns
y = df["mood"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features using StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Encode target variable using LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
print(y_train)
y_test = le.transform(y_test)

# Reshape input data to fit RNN input shape
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

[5 4 4 ... 4 5 6]


In [38]:
# Define hyperparameters
lstm_size = 100
num_layers = 2
dropout = 0.2
optimizer = 'adam'
eps = 20
batch = 64

# Define RNN model architecture
model = Sequential()    
for i in range(num_layers):
    model.add(LSTM(units=lstm_size, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout))
    
model.add(LSTM(units=lstm_size))
model.add(Dropout(dropout))
model.add(Dense(units=10, activation='softmax'))

# Compile model
#model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=eps, batch_size=batch, validation_data=(X_test, y_test))

# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)

print('Test loss:', loss)
print('Test accuracy:', accuracy)



Epoch 1/20
17/17 [==============================] - 16s 220ms/step - loss: 2.2742 - accuracy: 0.4473 - val_loss: 2.2270 - val_accuracy: 0.5517
Epoch 2/20
17/17 [==============================] - 0s 25ms/step - loss: 2.1570 - accuracy: 0.5268 - val_loss: 2.0000 - val_accuracy: 0.5517
Epoch 3/20
17/17 [==============================] - 0s 21ms/step - loss: 1.7524 - accuracy: 0.5259 - val_loss: 1.3758 - val_accuracy: 0.5402
Epoch 4/20
17/17 [==============================] - 0s 22ms/step - loss: 1.2722 - accuracy: 0.5038 - val_loss: 1.1077 - val_accuracy: 0.5556
Epoch 5/20
17/17 [==============================] - 0s 28ms/step - loss: 1.0844 - accuracy: 0.5556 - val_loss: 1.0332 - val_accuracy: 0.5479
Epoch 6/20
17/17 [==============================] - 0s 20ms/step - loss: 0.9902 - accuracy: 0.5776 - val_loss: 0.9741 - val_accuracy: 0.5747
Epoch 7/20
17/17 [==============================] - 0s 21ms/step - loss: 0.9185 - accuracy: 0.6322 - val_loss: 0.9126 - val_accuracy: 0.6130
Epoch 8/20


In [24]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


# Define function to create RNN model
def create_model(lstm_size=50, num_layers=2, dropout=0.5, optimizer='adam'):
    model = Sequential()
    for i in range(num_layers):
        model.add(LSTM(units=lstm_size, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(dropout))
    model.add(LSTM(units=lstm_size))
    model.add(Dropout(dropout))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create KerasClassifier object
model = KerasClassifier(build_fn=create_model)

# Create dictionary for hyperparameters
param_grid = {
    'lstm_size': [50, 100, 150],
    'num_layers': [2, 3],
    'dropout': [0.1, 0.2, 0.3],
    'batch_size': [32, 64],
    'epochs': [20, 50],
    'optimizer': ['adam']
}

# Create GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

# Print best parameters and accuracy
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


C:\Users\noahv\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/20
22/22 [==============================] - 12s 100ms/step - loss: 2.2740 - accuracy: 0.4655 - val_loss: 2.2344 - val_accuracy: 0.5479
Epoch 2/20
22/22 [==============================] - 0s 16ms/step - loss: 2.1789 - accuracy: 0.5144 - val_loss: 2.0741 - val_accuracy: 0.5479
Epoch 3/20
22/22 [==============================] - 0s 17ms/step - loss: 1.8861 - accuracy: 0.5201 - val_loss: 1.5361 - val_accuracy: 0.5479
Epoch 4/20
22/22 [==============================] - 1s 31ms/step - loss: 1.3441 - accuracy: 0.5172 - val_loss: 1.1871 - val_accuracy: 0.5479
Epoch 5/20
22/22 [==============================] - 1s 31ms/step - loss: 1.1429 - accuracy: 0.5172 - val_loss: 1.1014 - val_accuracy: 0.5479
Epoch 6/20
22/22 [==============================] - 1s 31ms/step - loss: 1.0952 - accuracy: 0.5230 - val_loss: 1.0881 - val_accuracy: 0.5479
Epoch 7/20
22/22 [==============================] - 1s 31ms/step - loss: 1.0624 - accuracy: 0.5417 - val_loss: 1.0722 - val_accuracy: 0.5441
Epoch 8/20
